# Face Recognition with Siamese Networks

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import KFold, train_test_split

import wandb
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import models

import timm
from tqdm import tqdm
import logging
from rich.logging import RichHandler

import os
import time
import copy
import joblib
from collections import defaultdict
import gc
import random
from glob import glob

from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

%load_ext watermark
%watermark -v -p torch,matplotlib,numpy,seaborn,pandas

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.9.0

torch     : 2.0.0
matplotlib: 3.6.3
numpy     : 1.23.5
seaborn   : 0.12.2
pandas    : 1.5.3



# Config

In [2]:
class cfg:
    seed   = 42
    project_name = 'frs-kavach'
    exp_name = 'siamese-frs'
    base_model    = 'convnext_base'
    train_bs      = 8
    valid_bs      = 2 * train_bs
    image_size    = [112, 112]
    comment       = f'basemodel-{base_model}|img_size-{image_size[0]}x{image_size[1]}'
    epochs        = 10
    
    optimizer     = 'Adam'
    learning_rate = 3e-4
    rho           = 0.9
    eps           = 1e-6
    lr_decay      = 0
    betas         = (0.9, 0.999)
    momentum      = 0
    alpha         = 0.99
    
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-6
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 0
    weight_decay  = 1e-6
    
    n_accumulate  = max(1, 32//train_bs)
    num_folds     = 5
    num_classes   = None

    device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    iteration_num = 1

    train_path    = '../../artifacts/archive/'

# Utils

In [3]:
def set_seed(seed = 42):    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('>>> SEEDING DONE')
    
set_seed(cfg.seed)

>>> SEEDING DONE


In [4]:
def get_scheduler(optimizer: optim):
    '''
    A method which returns the required schedulers.
        - Extracted from Awsaf's Kaggle.
    '''
    if config.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer=optimizer, 
            T_max=config.T_max, 
            eta_min=config.min_lr
        )
    elif config.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer=optimizer, 
            T_0=config.T_0, 
            eta_min=config.eta_min
        )
    elif config.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer, 
            mode='min',
            factor=0.1, 
            patience=10, 
            threshold=0.0001, 
            min_lr=config.min_lr
        )
    elif config.scheduler == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(
            optimizer=optimizer, 
            gamma=0.85
        )
    elif config.scheduler is None:
        scheduler = None
    else:
        raise NotImplementedError("The Scheduler you have asked has not been implemented")
    return scheduler

In [6]:
def get_optimizer(model: nn.Module):
    '''
    Returns the optimizer based on the config files.
    '''
    if config.optimizer == 'Adadelta':
        optimizer = optim.Adadelta(
            model.parameters(), 
            lr=config.learning_rate,
            rho=config.rho, 
            eps=config.eps
        )
    elif config.optimizer == 'Adagrad':
        optimizer = optim.Adagrad(
            model.parameters(), 
            lr=config.learning_rate, 
            lr_decay=config.lr_decay,
            weight_decay=config.weight_decay
        )
    elif config.optimizer == 'Adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=config.learning_rate, 
            betas=config.betas, 
            eps=config.eps
        )
    elif config.optimizer == 'RMSProp':
        optimizer = optim.RMSprop(
            model.parameters(),
            lr=config.learning_rate, 
            alpha=config.alpha, 
            eps=config.eps, 
            weight_decay=config.weight_decay, 
            momentum=config.momentum
        )
    else:
        raise NotImplementedError(f"The optimizer {config.optimizer} has not been implemented.")
    return optimizer

# Data

In [28]:
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
        
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

# Plotting data
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:
class SiameseDataset(Dataset):
    def __init__(self, imgFo)